In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time

# Initialize MediaPipe Pose and drawing utilities
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# --- Utility Functions ---

def calculate_angle(a, b, c):
    """
    Calculates the angle between three 3D points (landmarks).
    """
    a = np.array(a)  # First point
    b = np.array(b)  # Mid point
    c = np.array(c)  # End point

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

def get_landmark_coords(landmarks, landmark_enum):
    """
    Returns the x, y coordinates of a specific landmark.
    """
    return [landmarks[landmark_enum.value].x, landmarks[landmark_enum.value].y]

# --- THE EXERCISE ENGINE ---
# This single function processes all exercises based on their configuration.

def process_exercise(landmarks, config, state):
    """
    Processes the current frame for a given exercise.
    """
    exercise_type = config.get('exercise_type', 'rep_based')

    # --- Posture Validation (optional) ---
    if 'posture_validation' in config:
        pv_config = config['posture_validation']
        p1 = get_landmark_coords(landmarks, pv_config['landmarks_to_track']['p1'])
        p2 = get_landmark_coords(landmarks, pv_config['landmarks_to_track']['p2'])
        p3 = get_landmark_coords(landmarks, pv_config['landmarks_to_track']['p3'])
        posture_angle = calculate_angle(p1, p2, p3)

        if posture_angle > pv_config['angle_threshold']:
            state['feedback'] = pv_config['feedback_map']['incorrect']
            state['color'] = (0, 0, 255)  # Red for incorrect posture
            return state  # Stop processing this frame

    # --- Main Exercise Logic ---

    # Repetition-based exercises (Squats, Curls, Push-ups, etc.)
    if exercise_type == 'rep_based':
        lm = config['landmarks_to_track']
        # Handles exercises with one or two angles to track
        p1, p2, p3 = get_landmark_coords(landmarks, lm['p1']), get_landmark_coords(landmarks, lm['p2']), get_landmark_coords(landmarks, lm['p3'])
        angle1 = calculate_angle(p1, p2, p3)
        
        angle_to_use = angle1
        if 'p4' in lm:
            p4, p5, p6 = get_landmark_coords(landmarks, lm['p4']), get_landmark_coords(landmarks, lm['p5']), get_landmark_coords(landmarks, lm['p6'])
            angle2 = calculate_angle(p4, p5, p6)
            angle_logic = config.get('angle_logic', 'average')
            if angle_logic == 'min': angle_to_use = min(angle1, angle2)
            elif angle_logic == 'max': angle_to_use = max(angle1, angle2)
            else: angle_to_use = (angle1 + angle2) / 2
        
        up_thresh = config['angle_thresholds']['up_threshold']
        down_thresh = config['angle_thresholds']['down_threshold']
        fb_map = config['feedback_map']

        if not config.get('invert_stages', False):
            if angle_to_use > up_thresh:
                state.update({'stage': "up", 'feedback': fb_map['up'], 'color': (245, 117, 66)})
            elif angle_to_use < down_thresh and state['stage'] == 'up':
                state.update({'stage': "down", 'feedback': fb_map['down'], 'color': (0, 255, 0)})
                state['counter'] += 1
            else:
                if state['stage'] == 'up': state['feedback'] = fb_map.get('transition_down', 'Go Lower')
                elif state['stage'] == 'down': state['feedback'] = fb_map.get('transition_up', 'Go Higher')
                state['color'] = (0, 0, 255)
        else: # Inverted logic (e.g., Push-ups)
            if angle_to_use < down_thresh:
                 state.update({'stage': "down", 'feedback': fb_map['down'], 'color': (245, 117, 66)})
            elif angle_to_use > up_thresh and state['stage'] == 'down':
                state.update({'stage': "up", 'feedback': fb_map['up'], 'color': (0, 255, 0)})
                state['counter'] += 1
            else:
                if state['stage'] == 'down': state['feedback'] = fb_map.get('transition_up', 'Go Higher')
                elif state['stage'] == 'up': state['feedback'] = fb_map.get('transition_down', 'Go Lower')
                state['color'] = (0, 0, 255)
    
    # Timed exercises (Plank, Wall Sit)
    elif exercise_type == 'timed':
        lm = config['landmarks_to_track']
        p1, p2, p3 = get_landmark_coords(landmarks, lm['p1']), get_landmark_coords(landmarks, lm['p2']), get_landmark_coords(landmarks, lm['p3'])
        angle = calculate_angle(p1, p2, p3)
        
        correct_thresh = config['angle_thresholds']['correct_form_threshold']
        is_correct = angle > correct_thresh if not config.get('invert_logic') else angle < correct_thresh

        if is_correct:
            if state['start_time'] is None: state['start_time'] = time.time()
            state.update({'stage': 'correct', 'feedback': config['feedback_map']['correct'], 'color': (0, 255, 0)})
            state['counter'] = time.time() - state['start_time']
        else:
            state.update({'stage': 'incorrect', 'feedback': config['feedback_map']['incorrect'], 'color': (0, 0, 255)})
            state['start_time'] = None

    # Custom logic for High Knees
    elif exercise_type == 'knee_height':
        left_knee_y, left_hip_y = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
        right_knee_y, right_hip_y = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y

        if (left_knee_y < left_hip_y or right_knee_y < right_hip_y) and state['stage'] == 'down':
            state.update({'stage': 'up', 'feedback': config['feedback_map']['up'], 'color': (0, 255, 0)})
            state['counter'] += 1
        elif left_knee_y > left_hip_y and right_knee_y > right_hip_y:
            state.update({'stage': 'down', 'feedback': config['feedback_map']['down'], 'color': (245, 117, 66)})

    # Custom logic for Pull-ups
    elif exercise_type == 'pull_up':
        nose_y = landmarks[mp_pose.PoseLandmark.NOSE.value].y
        bar_y = min(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y)
        
        if nose_y < bar_y and state['stage'] == 'down':
             state.update({'stage': 'up', 'feedback': config['feedback_map']['up'], 'color': (0, 255, 0)})
             state['counter'] += 1
        elif nose_y > bar_y:
            state.update({'stage': 'down', 'feedback': config['feedback_map']['down'], 'color': (245, 117, 66)})

    # Custom logic for Russian Twists
    elif exercise_type == 'russian_twist':
        left_shoulder = np.array(get_landmark_coords(landmarks, mp_pose.PoseLandmark.LEFT_SHOULDER))
        right_shoulder = np.array(get_landmark_coords(landmarks, mp_pose.PoseLandmark.RIGHT_SHOULDER))
        
        shoulders_vec = right_shoulder - left_shoulder
        angle_shoulders = np.degrees(np.arctan2(shoulders_vec[1], shoulders_vec[0]))

        left_thresh, right_thresh = config['thresholds']['left'], config['thresholds']['right']

        if angle_shoulders < left_thresh and state['twist_stage'] == 'center':
            state['twist_stage'] = 'left'
            state['feedback'] = config['feedback_map']['left']
        elif angle_shoulders > right_thresh and state['twist_stage'] == 'center':
            state['twist_stage'] = 'right'
            state['feedback'] = config['feedback_map']['right']
        elif left_thresh < angle_shoulders < right_thresh:
            if state['twist_stage'] == 'left': state['has_twisted_left'] = True
            elif state['twist_stage'] == 'right': state['has_twisted_right'] = True
            state['twist_stage'] = 'center'
            state['feedback'] = 'Center'
        
        if state['has_twisted_left'] and state['has_twisted_right']:
            state['counter'] += 1
            state['feedback'] = 'Good Rep!'
            state['has_twisted_left'], state['has_twisted_right'] = False, False

    # Custom logic for Bird-Dog
    elif exercise_type == 'bird_dog':
        left_wrist = np.array(get_landmark_coords(landmarks, mp_pose.PoseLandmark.LEFT_WRIST))
        right_knee = np.array(get_landmark_coords(landmarks, mp_pose.PoseLandmark.RIGHT_KNEE))
        dist_lr = np.linalg.norm(left_wrist - right_knee)
        
        if dist_lr > config['thresholds']['extended'] and state['stage'] == 'in':
            state.update({'stage': 'out', 'feedback': config['feedback_map']['out']})
            state['counter'] += 1
        elif dist_lr < config['thresholds']['contracted']:
            state.update({'stage': 'in', 'feedback': config['feedback_map']['in']})
            
    # Custom logic for Mountain Climbers
    elif exercise_type == 'mountain_climber':
        left_knee = np.array(get_landmark_coords(landmarks, mp_pose.PoseLandmark.LEFT_KNEE))
        left_elbow = np.array(get_landmark_coords(landmarks, mp_pose.PoseLandmark.LEFT_ELBOW))
        dist = np.linalg.norm(left_knee - left_elbow)

        if dist < config['thresholds']['close'] and state['stage'] == 'back':
            state.update({'stage': 'forward', 'feedback': config['feedback_map']['forward']})
            state['counter'] +=1
        elif dist > config['thresholds']['far']:
            state.update({'stage': 'back', 'feedback': config['feedback_map']['back']})
            
    # Custom logic for Burpees (State Machine)
    elif exercise_type == 'burpee':
        hip_l, knee_l, ankle_l = get_landmark_coords(landmarks, mp_pose.PoseLandmark.LEFT_HIP), get_landmark_coords(landmarks, mp_pose.PoseLandmark.LEFT_KNEE), get_landmark_coords(landmarks, mp_pose.PoseLandmark.LEFT_ANKLE)
        shoulder_l, hip_l_2, elbow_l, wrist_l = get_landmark_coords(landmarks, mp_pose.PoseLandmark.LEFT_SHOULDER), get_landmark_coords(landmarks, mp_pose.PoseLandmark.LEFT_HIP), get_landmark_coords(landmarks, mp_pose.PoseLandmark.LEFT_ELBOW), get_landmark_coords(landmarks, mp_pose.PoseLandmark.LEFT_WRIST)
        
        squat_angle = calculate_angle(hip_l, knee_l, ankle_l)
        plank_angle = calculate_angle(shoulder_l, hip_l_2, ankle_l)
        pushup_angle = calculate_angle(shoulder_l, elbow_l, wrist_l)

        if state['burpee_stage'] == 'start' and squat_angle < 100: state.update({'burpee_stage': 'squat', 'feedback': 'Down to Plank'})
        elif state['burpee_stage'] == 'squat' and plank_angle > 160: state.update({'burpee_stage': 'plank', 'feedback': 'Push-up'})
        elif state['burpee_stage'] == 'plank' and pushup_angle < 90: state.update({'burpee_stage': 'pushup', 'feedback': 'Back to Squat'})
        elif state['burpee_stage'] == 'pushup' and squat_angle < 100 and plank_angle < 150: state.update({'burpee_stage': 'return_squat', 'feedback': 'Jump Up!'})
        elif state['burpee_stage'] == 'return_squat' and squat_angle > 165:
            state.update({'burpee_stage': 'start', 'counter': state['counter'] + 1, 'feedback': 'Good Rep!'})
            
    # Custom logic for Plank Jacks
    elif exercise_type == 'plank_jacks':
        left_ankle_x = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x
        right_ankle_x = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x
        ankle_dist = abs(left_ankle_x - right_ankle_x)

        if ankle_dist > config['thresholds']['out'] and state['stage'] == 'in':
            state.update({'stage': 'out', 'feedback': config['feedback_map']['out']})
            state['counter'] += 1
        elif ankle_dist < config['thresholds']['in']:
            state.update({'stage': 'in', 'feedback': config['feedback_map']['in']})
            
    # Custom logic for Shoulder Taps
    elif exercise_type == 'shoulder_taps':
        left_wrist = np.array(get_landmark_coords(landmarks, mp_pose.PoseLandmark.LEFT_WRIST))
        right_shoulder = np.array(get_landmark_coords(landmarks, mp_pose.PoseLandmark.RIGHT_SHOULDER))
        dist = np.linalg.norm(left_wrist - right_shoulder)

        if dist < config['thresholds']['tap'] and state['stage'] == 'down':
            state.update({'stage': 'up', 'feedback': config['feedback_map']['tap']})
            state['counter'] += 1
        elif dist > config['thresholds']['release']:
            state.update({'stage': 'down', 'feedback': config['feedback_map']['release']})

    return state


# --- Exercise Configurations ---
# Each exercise is defined by a dictionary. This list now contains 50 exercises.
exercises = [
    {'name': 'SQUAT','landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_HIP, 'p2': mp_pose.PoseLandmark.LEFT_KNEE, 'p3': mp_pose.PoseLandmark.LEFT_ANKLE, 'p4': mp_pose.PoseLandmark.RIGHT_HIP, 'p5': mp_pose.PoseLandmark.RIGHT_KNEE, 'p6': mp_pose.PoseLandmark.RIGHT_ANKLE },'angle_thresholds': { 'up_threshold': 165, 'down_threshold': 90 },'feedback_map': { 'up': 'Ready', 'down': 'Good Squat!', 'transition_down': 'Go Deeper', 'transition_up': 'Push Up' }, 'angle_logic':'min'},
    {'name': 'BICEP CURL','landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_ELBOW, 'p3': mp_pose.PoseLandmark.LEFT_WRIST, 'p4': mp_pose.PoseLandmark.RIGHT_SHOULDER, 'p5': mp_pose.PoseLandmark.RIGHT_ELBOW, 'p6': mp_pose.PoseLandmark.RIGHT_WRIST },'angle_thresholds': { 'up_threshold': 160, 'down_threshold': 40 },'feedback_map': { 'up': 'Ready', 'down': 'Good Rep', 'transition_down': 'Curl Up', 'transition_up': 'Lower Slowly' }, 'angle_logic':'min'},
    {'name': 'PUSH-UP', 'invert_stages': True, 'landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_ELBOW, 'p3': mp_pose.PoseLandmark.LEFT_WRIST, 'p4': mp_pose.PoseLandmark.RIGHT_SHOULDER, 'p5': mp_pose.PoseLandmark.RIGHT_ELBOW, 'p6': mp_pose.PoseLandmark.RIGHT_WRIST },'angle_thresholds': { 'up_threshold': 160, 'down_threshold': 90 },'feedback_map': { 'up': 'Good Rep', 'down': 'Ready', 'transition_down': 'Go Lower', 'transition_up': 'Push Up' }, 'angle_logic':'min'},
    {'name': 'LUNGE','landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_HIP, 'p2': mp_pose.PoseLandmark.LEFT_KNEE, 'p3': mp_pose.PoseLandmark.LEFT_ANKLE, 'p4': mp_pose.PoseLandmark.RIGHT_HIP, 'p5': mp_pose.PoseLandmark.RIGHT_KNEE, 'p6': mp_pose.PoseLandmark.RIGHT_ANKLE },'angle_thresholds': { 'up_threshold': 160, 'down_threshold': 100 },'feedback_map': { 'up': 'Ready', 'down': 'Good Rep', 'transition_down': 'Step Forward', 'transition_up': 'Push Back Up' }, 'angle_logic':'min'},
    {'name': 'PLANK', 'exercise_type': 'timed','landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_ANKLE },'angle_thresholds': { 'correct_form_threshold': 160 },'feedback_map': { 'correct': 'Hold Position', 'incorrect': 'Straighten Back' }},
    {'name': 'OVERHEAD PRESS','invert_stages': True,'landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_ELBOW, 'p3': mp_pose.PoseLandmark.LEFT_WRIST, 'p4': mp_pose.PoseLandmark.RIGHT_SHOULDER, 'p5': mp_pose.PoseLandmark.RIGHT_ELBOW, 'p6': mp_pose.PoseLandmark.RIGHT_WRIST },'angle_thresholds': { 'up_threshold': 160, 'down_threshold': 90 },'feedback_map': { 'up': 'Good Rep', 'down': 'Ready', 'transition_down': 'Lower Slowly', 'transition_up': 'Press Up' }, 'angle_logic':'min'},
    {'name': 'JUMPING JACKS','landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_HIP, 'p2': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p3': mp_pose.PoseLandmark.LEFT_WRIST, 'p4': mp_pose.PoseLandmark.RIGHT_HIP, 'p5': mp_pose.PoseLandmark.RIGHT_SHOULDER, 'p6': mp_pose.PoseLandmark.RIGHT_WRIST },'angle_thresholds': { 'up_threshold': 130, 'down_threshold': 50 },'feedback_map': { 'up': 'Good Rep', 'down': 'Ready', 'transition_down': 'Arms Up!', 'transition_up': 'Arms Down' }, 'angle_logic':'min'},
    {'name': 'GLUTE BRIDGE','invert_stages': True, 'landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_KNEE, 'p4': mp_pose.PoseLandmark.RIGHT_SHOULDER, 'p5': mp_pose.PoseLandmark.RIGHT_HIP, 'p6': mp_pose.PoseLandmark.RIGHT_KNEE },'angle_thresholds': { 'up_threshold': 160, 'down_threshold': 120 },'feedback_map': { 'up': 'Good Squeeze', 'down': 'Ready', 'transition_down': 'Lower Slowly', 'transition_up': 'Lift Hips' }, 'angle_logic':'min'},
    {'name': 'BENT OVER ROW','landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_ELBOW, 'p3': mp_pose.PoseLandmark.LEFT_WRIST, 'p4': mp_pose.PoseLandmark.RIGHT_SHOULDER, 'p5': mp_pose.PoseLandmark.RIGHT_ELBOW, 'p6': mp_pose.PoseLandmark.RIGHT_WRIST },'angle_thresholds': { 'up_threshold': 160, 'down_threshold': 90 },'feedback_map': { 'up': 'Ready', 'down': 'Good Squeeze', 'transition_down': 'Pull!', 'transition_up': 'Lower Slowly'},'posture_validation': {'landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_KNEE },'angle_threshold': 110, 'feedback_map': { 'incorrect': 'Bend Over More' } }, 'angle_logic':'min'},
    {'name': 'TRICEP DIPS', 'invert_stages': True, 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_ELBOW, 'p3': mp_pose.PoseLandmark.LEFT_WRIST, 'p4': mp_pose.PoseLandmark.RIGHT_SHOULDER, 'p5': mp_pose.PoseLandmark.RIGHT_ELBOW, 'p6': mp_pose.PoseLandmark.RIGHT_WRIST}, 'angle_thresholds': {'up_threshold': 160, 'down_threshold': 90}, 'feedback_map': {'up': 'Good Press', 'down': 'Ready', 'transition_down': 'Go Lower', 'transition_up': 'Push Up'}, 'angle_logic':'min'},
    {'name': 'CALF RAISES', 'invert_stages': True, 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_KNEE, 'p2': mp_pose.PoseLandmark.LEFT_ANKLE, 'p3': mp_pose.PoseLandmark.LEFT_HEEL, 'p4': mp_pose.PoseLandmark.RIGHT_KNEE, 'p5': mp_pose.PoseLandmark.RIGHT_ANKLE, 'p6': mp_pose.PoseLandmark.RIGHT_HEEL}, 'angle_thresholds': {'up_threshold': 170, 'down_threshold': 150}, 'feedback_map': {'up': 'Good Squeeze', 'down': 'Ready', 'transition_down': 'Lower Slowly', 'transition_up': 'Lift Heels'}, 'angle_logic':'min'},
    {'name': 'WALL SIT', 'exercise_type': 'timed', 'invert_logic': True, 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_HIP, 'p2': mp_pose.PoseLandmark.LEFT_KNEE, 'p3': mp_pose.PoseLandmark.LEFT_ANKLE}, 'angle_thresholds': {'correct_form_threshold': 120}, 'feedback_map': {'correct': 'Hold Tight!', 'incorrect': 'Get Lower!'}},
    {'name': 'DEADLIFT', 'invert_stages': True, 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_KNEE, 'p4': mp_pose.PoseLandmark.RIGHT_SHOULDER, 'p5': mp_pose.PoseLandmark.RIGHT_HIP, 'p6': mp_pose.PoseLandmark.RIGHT_KNEE}, 'angle_thresholds': {'up_threshold': 170, 'down_threshold': 90}, 'feedback_map': {'up': 'Good Rep!', 'down': 'Ready', 'transition_down': 'Hinge at Hips', 'transition_up': 'Extend Hips'}, 'angle_logic':'min'},
    {'name': 'HIGH KNEES', 'exercise_type': 'knee_height','feedback_map': {'up': 'Good!', 'down': 'Drive Knee Up!'}},
    {'name': 'PULL-UPS', 'exercise_type': 'pull_up', 'feedback_map': {'up': 'Good Rep!', 'down': 'Pull Up!'}},
    {'name': 'BIRD-DOG', 'exercise_type': 'bird_dog', 'thresholds': {'extended': 0.6, 'contracted': 0.2}, 'feedback_map': {'out': 'Extend!', 'in': 'Return'}},
    {'name': 'RUSSIAN TWIST', 'exercise_type': 'russian_twist', 'thresholds': {'left': -10, 'right': 10}, 'feedback_map': {'left': 'Twist Left', 'right': 'Twist Right'}},
    {'name': 'CRUNCHES', 'invert_stages': False, 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_KNEE}, 'angle_thresholds': {'up_threshold': 160, 'down_threshold': 130}, 'feedback_map': {'up': 'Ready', 'down': 'Good Crunch', 'transition_down': 'Crunch Up', 'transition_up': 'Lower Down'}},
    {'name': 'LEG RAISES', 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_ANKLE}, 'angle_thresholds': {'up_threshold': 150, 'down_threshold': 90}, 'feedback_map': {'up': 'Ready', 'down': 'Good Rep', 'transition_down': 'Raise Legs', 'transition_up': 'Lower Slowly'}},
    {'name': 'MOUNTAIN CLIMBER', 'exercise_type': 'mountain_climber', 'thresholds': {'close': 0.2, 'far': 0.4}, 'feedback_map': {'forward': 'Knee to Elbow!', 'back': 'Switch'}},
    {'name': 'SIDE LUNGES', 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_HIP, 'p2': mp_pose.PoseLandmark.LEFT_KNEE, 'p3': mp_pose.PoseLandmark.LEFT_ANKLE}, 'angle_thresholds': {'up_threshold': 160, 'down_threshold': 110}, 'feedback_map': {'up': 'Ready', 'down': 'Good Lunge', 'transition_down': 'Lunge Out', 'transition_up': 'Push Back'}},
    {'name': 'SUPERMAN', 'invert_stages': False, 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_ANKLE, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_SHOULDER}, 'angle_thresholds': {'up_threshold': 170, 'down_threshold': 150}, 'feedback_map': {'up': 'Ready', 'down': 'Lift!', 'transition_down': 'Lift!', 'transition_up': 'Lower Slowly'}},
    {'name': 'BURPEES', 'exercise_type': 'burpee', 'feedback_map': {}},
    {'name': 'SIDE PLANK', 'exercise_type': 'timed', 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_ANKLE, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_SHOULDER}, 'angle_thresholds': {'correct_form_threshold': 150}, 'feedback_map': {'correct': 'Hold Straight!', 'incorrect': 'Lift Hips!'}},
    {'name': 'LATERAL RAISES', 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_HIP, 'p2': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p3': mp_pose.PoseLandmark.LEFT_ELBOW, 'p4': mp_pose.PoseLandmark.RIGHT_HIP, 'p5': mp_pose.PoseLandmark.RIGHT_SHOULDER, 'p6': mp_pose.PoseLandmark.RIGHT_ELBOW}, 'angle_thresholds': {'up_threshold': 90, 'down_threshold': 20}, 'feedback_map': {'up': 'Ready', 'down': 'Good Rep', 'transition_down': 'Raise Arms', 'transition_up': 'Lower Slowly'}, 'angle_logic': 'max'},
    {'name': 'SUMO SQUAT', 'landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_HIP, 'p2': mp_pose.PoseLandmark.LEFT_KNEE, 'p3': mp_pose.PoseLandmark.LEFT_ANKLE, 'p4': mp_pose.PoseLandmark.RIGHT_HIP, 'p5': mp_pose.PoseLandmark.RIGHT_KNEE, 'p6': mp_pose.PoseLandmark.RIGHT_ANKLE },'angle_thresholds': { 'up_threshold': 165, 'down_threshold': 80 },'feedback_map': { 'up': 'Ready', 'down': 'Good Squat!', 'transition_down': 'Go Deeper', 'transition_up': 'Push Up' }, 'angle_logic':'min'},
    {'name': 'PIKE PUSH-UP', 'invert_stages': True, 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_ELBOW, 'p3': mp_pose.PoseLandmark.LEFT_WRIST}, 'angle_thresholds': {'up_threshold': 160, 'down_threshold': 100}, 'feedback_map': {'up': 'Good Rep', 'down': 'Ready', 'transition_down': 'Lower Head', 'transition_up': 'Press Up'}},
    {'name': 'REVERSE CRUNCHES', 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_KNEE}, 'angle_thresholds': {'up_threshold': 120, 'down_threshold': 80}, 'feedback_map': {'up': 'Ready', 'down': 'Good Rep', 'transition_down': 'Knees to Chest', 'transition_up': 'Lower Legs'}},
    {'name': 'PLANK JACKS', 'exercise_type': 'plank_jacks', 'thresholds': {'out': 0.4, 'in': 0.2}, 'feedback_map': {'out': 'Legs Out!', 'in': 'Legs In!'}},
    {'name': 'GOOD MORNINGS', 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_KNEE}, 'angle_thresholds': {'up_threshold': 170, 'down_threshold': 100}, 'feedback_map': {'up': 'Ready', 'down': 'Good Hinge', 'transition_down': 'Hinge Forward', 'transition_up': 'Squeeze Glutes'}},
    {'name': 'DONKEY KICKS', 'invert_stages': True, 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_ANKLE}, 'angle_thresholds': {'up_threshold': 120, 'down_threshold': 90}, 'feedback_map': {'up': 'Good Kick!', 'down': 'Ready', 'transition_up': 'Kick Up', 'transition_down': 'Return'}},
    {'name': 'FIRE HYDRANTS', 'invert_stages': True, 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_HIP, 'p2': mp_pose.PoseLandmark.LEFT_KNEE, 'p3': mp_pose.PoseLandmark.RIGHT_KNEE}, 'angle_thresholds': {'up_threshold': 100, 'down_threshold': 90}, 'feedback_map': {'up': 'Good Lift!', 'down': 'Ready', 'transition_up': 'Lift Knee', 'transition_down': 'Return'}},
    {'name': 'SHOULDER TAPS', 'exercise_type': 'shoulder_taps', 'thresholds': {'tap': 0.1, 'release': 0.2}, 'feedback_map': {'tap': 'Tap!', 'release': 'Return Hand'}},
    {'name': 'WALL PUSH-UPS', 'invert_stages': True, 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_ELBOW, 'p3': mp_pose.PoseLandmark.LEFT_WRIST}, 'angle_thresholds': {'up_threshold': 160, 'down_threshold': 90}, 'feedback_map': {'up': 'Good Press', 'down': 'Ready', 'transition_up': 'Push Away', 'transition_down': 'Lean In'}},
    {'name': 'ARM CIRCLES', 'exercise_type': 'timed', 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_ELBOW, 'p2': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p3': mp_pose.PoseLandmark.LEFT_HIP}, 'angle_thresholds': {'correct_form_threshold': 30}, 'feedback_map': {'correct': 'Keep Circling', 'incorrect': 'Raise Arms'}},
    {'name': 'TORSO TWISTS', 'exercise_type': 'russian_twist', 'thresholds': {'left': -5, 'right': 5}, 'feedback_map': {'left': 'Twist Left', 'right': 'Twist Right'}},
    {'name': 'REVERSE LUNGES', 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_HIP, 'p2': mp_pose.PoseLandmark.LEFT_KNEE, 'p3': mp_pose.PoseLandmark.LEFT_ANKLE}, 'angle_thresholds': {'up_threshold': 160, 'down_threshold': 100}, 'feedback_map': {'up': 'Ready', 'down': 'Good Lunge', 'transition_down': 'Step Back', 'transition_up': 'Return'}},
    {'name': 'FORWARD FOLD', 'exercise_type': 'timed', 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_KNEE}, 'angle_thresholds': {'correct_form_threshold': 160}, 'feedback_map': {'correct': 'Hold Stretch', 'incorrect': 'Straighten Back'}},
    {'name': 'CAT-COW STRETCH', 'invert_stages': True, 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_KNEE}, 'angle_thresholds': {'up_threshold': 100, 'down_threshold': 80}, 'feedback_map': {'up': 'Cow Pose', 'down': 'Cat Pose', 'transition_up': 'Arch Back', 'transition_down': 'Round Spine'}},
    {'name': "CHILD'S POSE", 'exercise_type': 'timed', 'invert_logic': True, 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_KNEE}, 'angle_thresholds': {'correct_form_threshold': 80}, 'feedback_map': {'correct': 'Hold and Breathe', 'incorrect': 'Sit Back on Heels'}},
    {'name': 'COBRA POSE', 'exercise_type': 'timed', 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_HIP, 'p2': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p3': mp_pose.PoseLandmark.LEFT_ELBOW}, 'angle_thresholds': {'correct_form_threshold': 150}, 'feedback_map': {'correct': 'Hold Pose', 'incorrect': 'Lift Chest'}},
    {'name': 'DOWNWARD DOG', 'exercise_type': 'timed', 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_ANKLE, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_WRIST}, 'angle_thresholds': {'correct_form_threshold': 150}, 'feedback_map': {'correct': 'Hold the V-Shape', 'incorrect': 'Push Hips Up'}},
    {'name': 'DIAMOND PUSH-UP', 'invert_stages': True, 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_ELBOW, 'p3': mp_pose.PoseLandmark.LEFT_WRIST}, 'angle_thresholds': {'up_threshold': 160, 'down_threshold': 90}, 'feedback_map': {'up': 'Good Rep', 'down': 'Ready', 'transition_down': 'Go Lower', 'transition_up': 'Push Up'}},
    {'name': 'FLUTTER KICKS', 'exercise_type': 'timed', 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_HIP, 'p2': mp_pose.PoseLandmark.LEFT_KNEE, 'p3': mp_pose.PoseLandmark.LEFT_ANKLE}, 'angle_thresholds': {'correct_form_threshold': 160}, 'feedback_map': {'correct': 'Keep Kicking', 'incorrect': 'Keep Legs Straight'}},
    {'name': 'SCISSOR KICKS', 'exercise_type': 'timed', 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_HIP, 'p2': mp_pose.PoseLandmark.LEFT_KNEE, 'p3': mp_pose.PoseLandmark.LEFT_ANKLE}, 'angle_thresholds': {'correct_form_threshold': 160}, 'feedback_map': {'correct': 'Keep Crossing', 'incorrect': 'Keep Legs Straight'}},
    {'name': 'INCHWORM', 'exercise_type': 'rep_based', 'invert_stages': True, 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_ANKLE}, 'angle_thresholds': {'up_threshold': 160, 'down_threshold': 90}, 'feedback_map': {'up': 'Walk Feet In', 'down': 'Walk Hands Out', 'transition_up': 'Walk Feet In', 'transition_down': 'Walk Hands Out'}},
    {'name': 'HIGH PLANK TO LOW PLANK', 'invert_stages': True, 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_ELBOW, 'p3': mp_pose.PoseLandmark.LEFT_WRIST}, 'angle_thresholds': {'up_threshold': 160, 'down_threshold': 90}, 'feedback_map': {'up': 'High Plank', 'down': 'Low Plank', 'transition_up': 'Up to Hands', 'transition_down': 'Down to Elbows'}},
    {'name': 'BOXER SHUFFLE', 'exercise_type': 'timed', 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_KNEE, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_SHOULDER}, 'angle_thresholds': {'correct_form_threshold': 150}, 'feedback_map': {'correct': 'Keep Shuffling', 'incorrect': 'Stay on Toes'}},
    {'name': 'SIDE BEND', 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_KNEE}, 'angle_thresholds': {'up_threshold': 170, 'down_threshold': 155}, 'feedback_map': {'up': 'Ready', 'down': 'Good Bend', 'transition_down': 'Bend Sideways', 'transition_up': 'Return to Center'}},
    {'name': 'T-POSE HOLD', 'exercise_type': 'timed', 'landmarks_to_track': {'p1': mp_pose.PoseLandmark.LEFT_HIP, 'p2': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p3': mp_pose.PoseLandmark.LEFT_WRIST}, 'angle_thresholds': {'correct_form_threshold': 160}, 'feedback_map': {'correct': 'Hold Strong!', 'incorrect': 'Straighten Arms'}}
]


# --- State Management ---
def reset_state(config):
    """
    Resets the state dictionary for a new exercise.
    """
    initial_stage = 'down' if config.get('invert_stages') else 'up'
    
    state = {'counter': 0, 'stage': initial_stage, 'feedback': 'Ready', 'color': (245, 117, 66), 'start_time': None}
    
    if config.get('exercise_type') == 'timed':
        state['stage'] = 'start'
        state['feedback'] = 'Get Ready'
    elif config.get('exercise_type') == 'burpee':
        state['burpee_stage'] = 'start' # Custom state for burpee state machine
    elif config.get('exercise_type') == 'russian_twist':
        state['twist_stage'] = 'center'
        state['has_twisted_left'] = False
        state['has_twisted_right'] = False
        
    return state


# --- Main Application Loop ---

def main():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open video stream.")
        return

    # Set initial exercise
    current_exercise_index = 0
    current_exercise_config = exercises[current_exercise_index]
    state = reset_state(current_exercise_config)

    print("--- AI Fitness Coach ---")
    print("Press '.' for NEXT or ',' for PREVIOUS exercise. Press ESC to quit.")


    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        # Process frame
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image_rgb.flags.writeable = False # Performance optimization
        results = pose.process(image_rgb)
        image_rgb.flags.writeable = True

        # Exercise logic
        if results.pose_landmarks:
            state = process_exercise(results.pose_landmarks.landmark, current_exercise_config, state)
            # Render landmarks
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=state['color'], thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=state['color'], thickness=2, circle_radius=2))
        else:
            state['feedback'] = 'No Pose Detected'
            if current_exercise_config.get('exercise_type') == 'timed':
                state['start_time'] = None # Pause timer if pose is lost

        # Flip the frame horizontally for a selfie-view display.
        frame = cv2.flip(frame, 1)

        # --- Visualization UI ---
        # Draw UI box
        cv2.rectangle(frame, (15, 15), (620, 100), (0,0,0), -1)

        # Display Exercise Name
        cv2.putText(frame, 'EXERCISE', (25, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(frame, f"({current_exercise_index + 1}/{len(exercises)}) {current_exercise_config['name']}", (30, 85), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)

        # Display Counter
        counter_label = 'REPS' if 'timed' not in current_exercise_config.get('exercise_type', 'rep_based') else 'TIME (s)'
        display_counter = int(state['counter'])
        cv2.putText(frame, counter_label, (340, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(frame, str(display_counter), (345, 85), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)

        # Display Feedback
        cv2.putText(frame, 'FEEDBACK', (460, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
        feedback_text = state.get('feedback', 'Ready')
        (text_width, _), _ = cv2.getTextSize(feedback_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.putText(frame, feedback_text, (610 - text_width, 85), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2, cv2.LINE_AA)

        # Display the frame
        cv2.imshow('AI Fitness Coach', frame)

        # --- Key press logic ---
        # Use a slightly longer wait key to reduce CPU usage
        key = cv2.waitKey(10) & 0xFF
        
        # 'q' is already used for an exercise, so we use ESC to quit
        if key == 27: # 27 is the ESC key
            break
        
        # Check if the pressed key corresponds to an exercise
        elif key == ord('.'): # Next
            current_exercise_index = (current_exercise_index + 1) % len(exercises)
            current_exercise_config = exercises[current_exercise_index]
            state = reset_state(current_exercise_config)
            print(f"Switched to {current_exercise_config['name']}")

        elif key == ord(','): # Previous
            current_exercise_index = (current_exercise_index - 1 + len(exercises)) % len(exercises)
            current_exercise_config = exercises[current_exercise_index]
            state = reset_state(current_exercise_config)
            print(f"Switched to {current_exercise_config['name']}")

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()



--- AI Fitness Coach ---
Press '.' for NEXT or ',' for PREVIOUS exercise. Press ESC to quit.
Switched to T-POSE HOLD
Switched to SIDE BEND
Switched to BOXER SHUFFLE
